In [48]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta

In [49]:
df = pd.read_excel('Portfolio_db\stock_portfolio2.xlsx', header=1)
print(df)

    Unnamed: 0      Mercado de Capitales Nacional Unnamed: 2      Unnamed: 3  \
0          NaN                      Emisora/Fondo    Títulos  Costo promedio   
1          NaN                           CEMEXCPO        100           28.16   
2          NaN                             WALMEX        120           78.53   
3          NaN  Mercado de Capitales Global (SIC)        NaN             NaN   
4          NaN                      Emisora/Fondo    Títulos  Costo promedio   
5          NaN                              LIN *          9             507   
6          NaN                             GOOG *         10            1838   
7          NaN                             META *         10            4346   
8          NaN                             NFLX *         14            1923   
9          NaN                             AMZN *          5            4943   
10         NaN                             TSLA *          5            2105   
11         NaN                          

In [50]:
# get original table and create a new table with relevant metrics only
df = pd.read_excel('Portfolio_db\stock_portfolio2.xlsx', header=1)
df_section = df.iloc[:, 1:]
sections = df_section[df_section.iloc[:, 0] == 'Emisora/Fondo'].index
headers = dict(df_section.iloc[sections[0],:])

data_nacional1 = df_section.iloc[sections[0]:sections[1]-1,:].rename(columns=headers).drop(0)
data_nacional1['Market'] = 'Nacional'

data_SIC1 = df_section.iloc[sections[1]:sections[2]-1,:].rename(columns=headers).drop(len(data_nacional1)+2)
data_SIC1['Market'] = 'SIC'

data_cash1 = pd.DataFrame(df_section.iloc[sections[2]:,:].rename(columns=headers).drop(len(data_nacional1)+len(data_SIC1)+4).sum()).transpose()
data_cash1.at[0, 'Emisora/Fondo'] = 'Efectivo'
data_cash1['Market'] = 'Efectivo'

data_port1 = pd.concat([data_nacional1, data_SIC1, data_cash1], ignore_index=True)[['Emisora/Fondo', 'Valor mercado', 'Market', 'P / M']]
data_port1['Emisora/Fondo'] = data_port1['Emisora/Fondo'].apply(lambda x: x.replace(' *', '').strip() if isinstance(x, str) else x)
data_port1.rename(columns={'Emisora/Fondo': 'Ticker'}, inplace=True)


In [51]:
# Create function to retrieve information from Yahoo Finance and store it into a dictionary as output
def GetData(symbol):
    stock = yf.Ticker(symbol)

    try:
        # Get information and store it into variables
        industry = stock.info.get("industry", None)
        sector = stock.info.get("sector", None)
        trailingPE = round(float(stock.info["trailingPE"]),1) if "trailingPE" in stock.info and stock.info["trailingPE"] is not None else None
        forwardPE = round(float(stock.info["forwardPE"]),1) if "forwardPE" in stock.info and stock.info["forwardPE"] is not None else None
        bookValue = round(float(stock.info["bookValue"]),1) if "bookValue" in stock.info and stock.info["bookValue"] is not None else None
        priceToBook = round(float(stock.info["priceToBook"]),1) if "priceToBook" in stock.info and stock.info["priceToBook"] is not None else None
        recommendationKey = stock.info.get("recommendationKey", None)
        targetHighPrice = round(float(stock.info["targetHighPrice"]),3) if "targetHighPrice" in stock.info and stock.info["targetHighPrice"] is not None else 0
        targetMeanPrice = round(float(stock.info["targetMeanPrice"]),3) if "targetMeanPrice" in stock.info and stock.info["targetMeanPrice"] is not None else 0
        targetLowPrice = round(float(stock.info["targetLowPrice"]),3) if "targetLowPrice" in stock.info and stock.info["targetLowPrice"] is not None else 0
        
        #Create dictionary based on previous variables and return it
        stock_info = {
            'symbol': symbol,
            'industry': industry,
            'sector': sector,
            'trailing_PE': trailingPE,
            'forward_PE': forwardPE,
            'book_Value': bookValue,
            'price_To_Book': priceToBook,
            'recommendation_Key': recommendationKey,
            'target_High_Price': targetHighPrice,
            'target_Mean_Price': targetMeanPrice,
            'target_Low_Price': targetLowPrice
            }
            
        return stock_info
    
    except:
        # In case the stock cannot be found in yahoo library, return the same structure but with Null values
        stock_info = {
            'symbol': symbol,
            'industry': None,
            'sector': None,
            'trailing_PE': None,
            'forward_PE': None,
            'book_Value': None,
            'price_To_Book': None,
            'recommendation_Key': None,
            'target_High_Price': 0,
            'target_Mean_Price': 0,
            'target_Low_Price': 0
            }
        return stock_info


print(GetData('OPEN'))

{'symbol': 'OPEN', 'industry': 'Real Estate Services', 'sector': 'Real Estate', 'trailing_PE': None, 'forward_PE': -5.1, 'book_Value': 1.5, 'price_To_Book': 1.8, 'recommendation_Key': 'hold', 'target_High_Price': 4.5, 'target_Mean_Price': 2.52, 'target_Low_Price': 1.0}


In [52]:
#apply function to each value in Ticker column from data_port1 table 
data = pd.DataFrame(data_port1['Ticker'].apply(GetData).tolist())

print(data)

      symbol                        industry                  sector  \
0   CEMEXCPO                            None                    None   
1     WALMEX                            None                    None   
2        LIN             Specialty Chemicals         Basic Materials   
3       GOOG  Internet Content & Information  Communication Services   
4       META  Internet Content & Information  Communication Services   
5       NFLX                   Entertainment  Communication Services   
6       AMZN                 Internet Retail       Consumer Cyclical   
7       TSLA              Auto Manufacturers       Consumer Cyclical   
8        WMT                 Discount Stores      Consumer Defensive   
9         PG   Household & Personal Products      Consumer Defensive   
10      COST                 Discount Stores      Consumer Defensive   
11       CVX            Oil & Gas Integrated                  Energy   
12       SLB  Oil & Gas Equipment & Services                  En

In [53]:
#Merge the previous table with the fundamental information of each stock with the original dataset to consider Valor mercado, Market and P / M columns
final_data = data.merge(data_port1, left_on = 'symbol', right_on ='Ticker', how='right').drop(columns='Ticker')

print(final_data)

      symbol                        industry                  sector  \
0   CEMEXCPO                            None                    None   
1     WALMEX                            None                    None   
2        LIN             Specialty Chemicals         Basic Materials   
3       GOOG  Internet Content & Information  Communication Services   
4       META  Internet Content & Information  Communication Services   
5       NFLX                   Entertainment  Communication Services   
6       AMZN                 Internet Retail       Consumer Cyclical   
7       TSLA              Auto Manufacturers       Consumer Cyclical   
8        WMT                 Discount Stores      Consumer Defensive   
9         PG   Household & Personal Products      Consumer Defensive   
10      COST                 Discount Stores      Consumer Defensive   
11       CVX            Oil & Gas Integrated                  Energy   
12       SLB  Oil & Gas Equipment & Services                  En

In [54]:
#Get stock daily data

today = date.today()

endd = today.strftime("%Y-%m-%d")
startd = today - timedelta(days=365)
startdd = startd.strftime("%Y-%m-%d")


# Create function to retrieve data from each day of the last twelve months from yahoo finance 
def GetDataHistoric(symbol):

    try:
        h_data = yf.download(symbol, start=startdd, end=endd)[['High', 'Low', 'Volume']]
        h_data = h_data.copy()
        h_data.loc[:, 'Mean'] = (h_data['High'] + h_data['Low']) / 2
        h_data_stacked = h_data[['Mean', 'Volume']].reset_index()
        h_data_stacked['Ticker'] = symbol
        h_data_stacked.loc[:, 'Volume'] = h_data_stacked['Volume'].astype(int)
        return h_data_stacked

    except Exception as e:
        print(f"Error for {symbol}: {e}")


In [57]:
#Retrieve ticker historical data
historical_data = pd.DataFrame(columns=['Date', 'Mean', 'Volume','Ticker'])

for item in data_port1['Ticker']:
    try:
        historical_data = pd.concat([historical_data, GetDataHistoric(item)], ignore_index=True)
    except Exception as e:
        print(f"Error for {item}: {e}")

final_daily_data = pd.DataFrame(historical_data)
final_daily_data

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['CEMEXCPO']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['WALMEX']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******


1 Failed download:
['EFECTIVO']: Exception('%ticker%: No timezone found, symbol may be delisted')


,Date,Mean,Volume,Ticker
0,2022-11-28 00:00:00,340.86499,2881100,LIN
1,2022-11-29 00:00:00,333.255005,1888300,LIN
2,2022-11-30 00:00:00,332.73999,3115200,LIN
3,2022-12-01 00:00:00,335.520004,1420000,LIN
4,2022-12-02 00:00:00,336.294998,1200800,LIN
...,...,...,...,...
6245,2023-11-17 00:00:00,69.670002,4594500,SO
6246,2023-11-20 00:00:00,69.139999,3353000,SO
6247,2023-11-21 00:00:00,69.360001,3251400,SO
6248,2023-11-22 00:00:00,69.575001,2753000,SO
